# Data Collection and Preprocessing
This notebook focuses on data collection, cleaning, and preprocessing steps for the customer churn analysis.

## 1. Import Libraries

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import numpy as np

## 2. Data Collection

In [22]:
df = pd.read_csv(r'C:\Users\saife\OneDrive\Desktop\Graduation\data\raw\Customer-Churn-Records.csv')

In [23]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


## 3. Data Quality Checks

In [24]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   Surname             10000 non-null  object 
 3   CreditScore         10000 non-null  int64  
 4   Geography           10000 non-null  object 
 5   Gender              10000 non-null  object 
 6   Age                 10000 non-null  int64  
 7   Tenure              10000 non-null  int64  
 8   Balance             10000 non-null  float64
 9   NumOfProducts       10000 non-null  int64  
 10  HasCrCard           10000 non-null  int64  
 11  IsActiveMember      10000 non-null  int64  
 12  EstimatedSalary     10000 non-null  float64
 13  Exited              10000 non-null  int64  
 14  Complain            10000 non-null  int64  
 15  Satisfaction Score  10000 non-null  int64  
 16  Card 

(10000, 18)

In [25]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [26]:
duplicates = df.duplicated().sum()
print(f"\nNumber of Duplicate Rows: {duplicates}")


Number of Duplicate Rows: 0


In [27]:
df.isnull().sum()

CreditScore           0
Geography             0
Gender                0
Age                   0
Tenure                0
Balance               0
NumOfProducts         0
HasCrCard             0
IsActiveMember        0
EstimatedSalary       0
Exited                0
Complain              0
Satisfaction Score    0
Card Type             0
Point Earned          0
dtype: int64

In [28]:
# Check for outliers using IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers)

numeric_columns = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']
for col in numeric_columns:
    print(f'Outliers in {col}: {detect_outliers(df, col)}')

Outliers in CreditScore: 15
Outliers in Age: 359
Outliers in Balance: 0
Outliers in EstimatedSalary: 0


## 4. Feature Engineering

In [29]:
# Handle division by zero for feature creation
df['BalanceToSalary'] = df.apply(lambda x: 0 if x['EstimatedSalary'] == 0 else x['Balance'] / x['EstimatedSalary'], axis=1)
df['ProductsPerTenure'] = df.apply(lambda x: x['NumOfProducts'] if x['Tenure'] == 0 else x['NumOfProducts'] / x['Tenure'], axis=1)

# Replace infinite values with 0
df.replace([np.inf, -np.inf], 0, inplace=True)

# Scale numeric features
scaler = StandardScaler()
numeric_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 'BalanceToSalary', 'ProductsPerTenure']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [30]:
df = pd.get_dummies(df, columns=['Geography', 'Gender'], drop_first=True)

In [31]:
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,BalanceToSalary,ProductsPerTenure
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,10000.00000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04
mean,-4.824585e-16,2.318146e-16,-1.078249e-16,-6.252776e-17,1.530200,0.70550,0.515100,-2.877698e-17,0.203800,0.204400,3.013800,606.515100,4.973799e-18,1.957545e-16
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.581654,0.45584,0.499797,1.000050e+00,0.402842,0.403283,1.405919,225.924839,1.000050e+00,1.000050e+00
min,-3.109504e+00,-1.994969e+00,-1.733315e+00,-1.225848e+00,1.000000,0.00000,0.000000,-1.740268e+00,0.000000,0.000000,1.000000,119.000000,-3.580390e-02,-8.043445e-01
25%,-6.883586e-01,-6.600185e-01,-6.959818e-01,-1.225848e+00,1.000000,0.00000,0.000000,-8.535935e-01,0.000000,0.000000,2.000000,410.000000,-3.580390e-02,-6.074535e-01
50%,1.522218e-02,-1.832505e-01,-4.425957e-03,3.319639e-01,1.000000,1.00000,1.000000,1.802807e-03,0.000000,0.000000,3.000000,605.000000,-2.890840e-02,-3.449322e-01
75%,6.981094e-01,4.842246e-01,6.871299e-01,8.199205e-01,2.000000,1.00000,1.000000,8.572431e-01,0.000000,0.000000,4.000000,801.000000,-2.182812e-02,3.113709e-01
max,2.063884e+00,5.061197e+00,1.724464e+00,2.795323e+00,4.000000,1.00000,1.000000,1.737200e+00,1.000000,1.000000,5.000000,1000.000000,9.794698e+01,6.874403e+00


## 5. Save Processed Data

In [33]:
df.to_csv(r'C:\Users\saife\OneDrive\Desktop\Graduation\data\processed\data_after_preprocessing.csv', index=False)